In [1]:
import pandas as pd
import os
import datetime



In [2]:
master_df = pd.read_csv('tmax-2/tmax-2/part-00000.csv.gz',header=None, compression='gzip', sep=',', quotechar='"', error_bad_lines=False)
master_df.columns = ["station", "date", "latitude", "longitude","elevation","tmax"]



In [3]:
for file in os.listdir('tmax-2/tmax-2'):
    if(file != 'part-00000.csv.gz'):
        df = pd.read_csv('tmax-2/tmax-2/'+file, compression='gzip', header=None, sep=',', quotechar='"', error_bad_lines=False)
        df.columns = ["station", "date", "latitude", "longitude","elevation","tmax"]
        master_df = master_df.append(df, ignore_index = True)


master_df = master_df.dropna()
master_df

,station,date,latitude,longitude,elevation,tmax
0,USC00307895,1917-04-10,40.9000,-72.4500,11.0,5.0
1,RQC00668881,1986-07-10,18.3469,-67.0119,51.8,33.9
2,KNM00047014,2010-06-17,41.7830,126.8830,331.0,30.6
3,USC00306098,1896-05-14,43.8667,-75.1833,478.8,25.0
4,USC00272898,1963-08-29,44.2667,-71.4500,494.1,17.2
...,...,...,...,...,...,...
180659,FMW00040504,1967-10-05,6.9667,158.2167,36.6,31.1
180660,CA007086440,1960-03-01,47.9333,-78.5667,290.0,-11.1
180661,CA008500500,1901-10-19,51.8833,-55.3833,130.0,4.4
180662,USC00503504,2000-02-24,59.4503,-136.3614,249.9,0.0


In [4]:
master_df['date'] = master_df['date'].astype('datetime64[ns]') 
master_df['year_of_date'] = master_df['date'].dt.year

master_df['decade'] = master_df['date'].astype(str).str[:3].astype(int)*10





In [5]:

decade_1 = master_df[(master_df['decade'] >= 1910) & (master_df['decade'] <= 1960) ]
grouped_df_1 = decade_1.groupby(['station','latitude','longitude','elevation'],as_index=False).agg({'tmax':'mean'})

grouped_df_1

,station,latitude,longitude,elevation,tmax
0,AG000060590,30.5667,2.8667,397.0,17.229032
1,AG000060680,22.8000,5.4331,1362.0,29.806667
2,AGE00135039,35.7297,0.6500,50.0,25.064516
3,AGE00147713,36.1800,5.4000,1081.0,15.103448
4,AGM00060670,24.2930,9.4520,968.0,35.400000
...,...,...,...,...,...
2072,USW00094790,43.9922,-76.0217,96.9,11.290000
2073,UV000005522,10.3300,-3.1800,333.0,36.417857
2074,UZM00038457,41.2700,69.2694,477.0,23.726667
2075,UZM00038815,38.6170,66.2670,524.0,3.900000


In [6]:
decade_2 = master_df[(master_df['decade'] > 1960) & (master_df['decade'] <= 2010)]
grouped_decade_2 = decade_2.groupby(['station','latitude','longitude','elevation'],as_index=False).agg({'tmax':'mean'})



diff_df = pd.merge( grouped_decade_2, grouped_df_1, how='inner', on=['station','longitude','latitude','elevation'])
diff_df['temp_diff'] = diff_df['tmax_x']-diff_df['tmax_y']


diff_df


,station,latitude,longitude,elevation,tmax_x,tmax_y,temp_diff
0,ASN00009017,-32.0550,115.7500,5.0,30.560000,26.258065,4.301935
1,ASN00009534,-33.5719,115.8247,63.0,21.463158,22.596667,-1.133509
2,ASN00010111,-31.6508,116.6586,170.0,18.483333,17.716667,0.766667
3,ASN00022008,-34.3745,137.6733,185.0,28.885714,24.931034,3.954680
4,ASN00030045,-20.7289,143.1425,211.1,32.641935,26.767742,5.874194
...,...,...,...,...,...,...,...
297,USW00024284,43.4133,-124.2436,5.2,9.670968,18.423333,-8.752366
298,USW00026410,60.4889,-145.4511,9.4,7.354839,18.418519,-11.063680
299,USW00026617,64.5111,-165.4400,4.0,-5.946667,11.345161,-17.291828
300,USW00093026,31.4583,-109.6061,1251.2,13.616129,30.122581,-16.506452


In [20]:
import plotly.graph_objects as go
import plotly.express as px



layout=dict(title="Average Temperature Change in stations, between 1910 and 1960 and 1960 and 2010",
            geo=dict(landcolor='rgb(220,220,220)')
           )

fig = go.Figure(data=go.Scattergeo(lon = diff_df['longitude'],lat = diff_df['latitude'],mode = 'markers',
                                   marker={'showscale':True,'color':diff_df['temp_diff'],'cmin':diff_df['temp_diff'].min(),
                                           'cmax':diff_df['temp_diff'].max(),
                                           'colorbar':{'title':"Temperature Difference"}}),layout=layout)
fig.show()

In [8]:
from plotly.offline import plot

plot(fig, filename='Temperature_change.html')

'Temperature_change.html'

In [3]:
#for b1
import pandas as pd
import os
import datetime
import plotly
import plotly.express as px



In [4]:
#this file was generated from Spark and saved as a Csv
master_df = pd.read_csv('FinalFile.csv', sep=',', quotechar='"', error_bad_lines=False)

In [5]:
master_df

,latitude,longitude,elevation,tmax,date,prediction
0,-90.0,-180.0,0.0,0.0,25,26.767055
1,-89.5,-180.0,0.0,0.0,25,26.767055
2,-89.0,-180.0,0.0,0.0,25,26.767055
3,-88.5,-180.0,0.0,0.0,25,26.767055
4,-88.0,-180.0,0.0,0.0,25,26.767055
...,...,...,...,...,...,...
259195,87.5,179.5,0.0,0.0,25,-22.433163
259196,88.0,179.5,0.0,0.0,25,-22.433163
259197,88.5,179.5,0.0,0.0,25,-22.433163
259198,89.0,179.5,0.0,0.0,25,-22.433163


In [6]:
data = [dict(
    type='scattergeo',
    lon=master_df['longitude'],
    lat=master_df['latitude'],
    text=master_df['prediction'],
    mode="markers",
    marker=dict(
    color=master_df['prediction'],
    colorscale="temps",
    opacity=0.4,
    size=2,
        colorbar=dict(
        titleside="right",outlinecolor="rgba(68,68,68,0)")
    )
    )
]

layout = dict(
    title="Density heatmap showing the temperature at each latitude and longitude",
    geo = dict(
        scope="world",
        projection=dict(type="natural earth"),
        showland=True,
        landcolor="rgb(250,250,250)",
        subunitcolor="rgb(217,217,217)",
        countrycolor="rgb(217,217,217)",
        countrywidth=0.5,
        subunitwidth=0.5
    ),
)


In [8]:
fig = dict(data=data,layout=layout)
from plotly.offline import plot
#the maps take time load and may cause jupyter to crash, hence saving it as an html to open in a different browser window
plot(fig, filename='fig.html') 


'fig.html'

In [9]:
# showing the regression error (part b2)

# this file was generated using Spark and saved as a csv
testError_df = pd.read_csv('Test_error.csv', sep=',', quotechar='"', error_bad_lines=False)



In [10]:
data = [dict(
    type='scattergeo',
    lon=testError_df['longitude'],
    lat=testError_df['latitude'],
    text=testError_df['reg_error'],
    mode="markers",
    marker=dict(
    color=testError_df['prediction'],
    colorscale="temps",
    size=2,
        colorbar=dict(
        titleside="right",outlinecolor="rgba(68,68,68,0)")
    )
)]

layout=dict(
    title="Error between predicted tmax values and test tmax values",
     geo = dict(
        scope="world",
        projection=dict(type="equirectangular"),
         showland=True,
     ),
)

fig = dict(data=data,layout=layout)

from plotly.offline import plot
#the maps take time load and may cause jupyter to crash, hence saving it as an html to open in a different browser window
plot(fig, filename='Reg_Error.html') 

'Reg_Error.html'